In [ ]:
import pandas as pd
import requests
import json,math
import datetime as dt
import threading
import time
import numpy as np

# Function to extract data from url
def data_from_url (urls):
  
  df = pd.DataFrame()
  # url1 to store urls facing error_code 429 : Api limit exceeds  
  url1 = list()
  for url in urls:
    # Get response from url
    response = requests.get(url)
    time.sleep(0.10) # to prevent max api call per second
    if response.status_code == 200:  # check to make sure the response from server is good
            data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
            data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
            #if len(data.index) == 0:
             # Can print urls where for which coinbase doesn't have data
              # print(str(len(data.index))+ " " + url + '\n')
            df = df.append(data)
    
    # when error code = 429, Again call data_from_url
    #( if sleep is used above then no need of below code, still just for safer side )
    elif response.status_code == 429:
      # Store all missed API call
      print(f'{response.status_code} -- Error in 429 ' + url + '\n')
      url1.append(url)
      
    else:
        # Print url if status_code other than 200 or 429
        print("Did not receieve OK response from Coinbase API:" + url)
    
  if len(url1) > 0:
    data_from_url(url1)
  print('END of urls')
  return df      

# Function generated all urls for given currency
def generate_urls_from_symbol_name(range_end,symbol,epoch_startDate,epoch_endDate):
    # List to store all url for single symbol
    urls = []
    for i in range(0,range_end):
        # minutes = 300*interval
        url_endDate1 = int((dt.datetime.fromtimestamp(epoch_startDate) + dt.timedelta(minutes =300*5)).timestamp())
        
        #convert to readable format
        url_startDate = dt.datetime.utcfromtimestamp(epoch_startDate).strftime('%Y-%m-%dT%H:%M:%SZ')
        url_endDate = dt.datetime.utcfromtimestamp(url_endDate1).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        #below if else to handle edge case
        if epoch_endDate > url_endDate1:
            urls.append(f'https://api.pro.coinbase.com/products/{symbol}/candles?start={url_startDate}&end={url_endDate}&granularity=300')
            epoch_startDate = url_endDate1
        elif epoch_endDate < url_endDate1:
            epoch_endDate = dt.datetime.utcfromtimestamp(epoch_endDate).strftime('%Y-%m-%dT%H:%M:%SZ')
            urls.append(f'https://api.pro.coinbase.com/products/{symbol}/candles?start={url_startDate}&end={epoch_endDate}&granularity=300')
    #return all urls for single crypto_currency , e.g. returns 273 urls for range (6/1/2021 - 3/11/2022)
    return urls

def fetch_daily_data(pair,range_end,epoch_startDate,epoch_endDate,n):
   
  for p in pair:  
    #store all url
    urls = generate_urls_from_symbol_name(range_end,p,epoch_startDate,epoch_endDate)
    #extraxt data from urls
    data = data_from_url(urls)
    
    # if we failed to get any data, print an error...otherwise write the file
    if data is None:
            print("Did not return any data from Coinbase for this symbol:"+ p  )
    else:
            data.to_csv(f'Coinbase_{p}_dailydata.csv', index=False)
            # n is worker number
            print(f'Coinbase_{p}_dailydata.csv',n)

if __name__ == "__main__":
    # we set which pair we want to retrieve data for
    #pair = ["ZRX-USD","1INCH-USD","AAVE-USD"]
    pair = ["ZRX-USD","1INCH-USD","AAVE-USD","ALCX-USD","ACH-USD","AGLD-USD","ALGO-USD","AMP-USD","FORTH-USD","ANKR-USD","API3-USD","ARPA-USD",
            "ASM-USD","REP-USD","AVAX-USD","AVT-USD","AXS-USD","BADGER-USD","BAL-USD","BNT-USD","BAND-USD","BOND-USD","BAT-USD","BICO-USD",
            "BTC-USD","BCH-USD","BLZ-USD","FIDA-USD","AUCTION-USD","BTRST-USD","ADA-USD","CTSI-USD","CGLD-USD","LINK-USD","CHZ-USD","CVC-USD",
            "CLV-USD","COMP-USD","ATOM-USD","COTI-USD","COVAL-USD","CTX-USD","CRO-USD","CRV-USD","DAI-USD","DASH-USD","MANA-USD",
            "DESO-USD","DDX-USD","YFII-USD","DIA-USD","DNT-USD","DOGE-USD","ENJ-USD","MLN-USD","EOS-USD","ETH-USD","ETC-USD","ENS-USD",
            "FET-USD","FIL-USD","FX-USD","GALA-USD","GTC-USD","GODS-USD","GFI-USD","GYEN-USD","FARM-USD","ZEN-USD","IDEX-USD",
            "RLC-USD","IMX-USD","ICP-USD","INV-USD","IOTX-USD","JASMY-USD","KEEP-USD","KRL-USD","KNC-USD","LCX-USD","LQTY-USD","LTC-USD",
            "LPT-USD","LOOM-USD","LRC-USD","MKR-USD","MPL-USD","MASK-USD","MDT-USD","MIR-USD","MCO2-USD","MUSD-USD","NKN-USD","NU-USD",
            "NMR-USD","OMG-USD","ORCA-USD","OXT-USD","OGN-USD","TRAC-USD","ORN-USD","UPI-USD","PAX-USD","PERP-USD","PLA-USD",
            "PLU-USD","DOT-USD","POLS-USD","MATIC-USD","POLY-USD","NCT-USD","POWR-USD","PRO-USD","QNT-USD","QSP-USD","QUICK-USD","RAD-USD",
            "RAI-USD","RLY-USD","RGT-USD","RARI-USD","REN-USD","RNDR-USD","REQ-USD","RBN-USD","FOX-USD","SHIB-USD","SHPING-USD","SKL-USD",
            "SOL-USD","SPELL-USD","STX-USD","XLM-USD","STORJ-USD","SUKU-USD","SUPER-USD","SUSHI-USD","SNX-USD","TRB-USD",
            "UST-USD","USDT-USD","XTZ-USD","GRT-USD","TRIBE-USD","TRU-USD","UMA-USD","UNFI-USD","UNI-USD","VGX-USD","WBTC-USD",
            "wCFG-USD","WLUNA-USD","XYO-USD","YFI-USD","ZEC-USD"]
    epoch_startDate = dt.datetime(2021,6,1).timestamp()
    epoch_endDate = dt.datetime.today().timestamp()
    
    # (300*60*interval_in_minutes
    range_end = math.ceil((epoch_endDate-epoch_startDate)/(300*300))
    
    # Declare number of threads
    parallel_threads = 3
    #Splitting Pair files for each thread
    parallel_arrays = np.array_split(pair, parallel_threads)
    workers = list()
  
    start = dt.datetime.now()
    print(start)
    #Process Pair Files parallely
    for k in range(parallel_threads):
        # Implemented MultiThreading to process 3 threads in parallel
        wp = threading.Thread(target=fetch_daily_data, name='worker%d' % (k + 1), args=(list(parallel_arrays[k]),range_end,epoch_startDate,epoch_endDate,k+1,))
        workers.append(wp)
        wp.start()
   
    #Waitng for threads to complete
    for wp in workers:
        wp.join()
    end = dt.datetime.now()
    # Print total Time to extract Data
    print('END of program')
    print(start, end , end-start)

2022-03-12 14:45:49.859214
429 -- Error in 429 https://api.pro.coinbase.com/products/PLA-USD/candles?start=2021-06-21T00:00:00Z&end=2021-06-22T01:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/DOGE-USD/candles?start=2021-06-21T00:00:00Z&end=2021-06-22T01:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLA-USD/candles?start=2021-06-22T01:00:00Z&end=2021-06-23T02:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/DOGE-USD/candles?start=2021-06-23T02:00:00Z&end=2021-06-24T03:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLA-USD/candles?start=2021-06-23T02:00:00Z&end=2021-06-24T03:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLA-USD/candles?start=2021-06-25T04:00:00Z&end=2021-06-26T05:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLA-USD/candles?start=2021-06-26T05:00:00Z&end=2021-06-

429 -- Error in 429 https://api.pro.coinbase.com/products/PLA-USD/candles?start=2021-10-02T03:00:00Z&end=2021-10-03T04:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/DOGE-USD/candles?start=2021-09-26T22:00:00Z&end=2021-09-27T23:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLA-USD/candles?start=2021-10-17T18:00:00Z&end=2021-10-18T19:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLA-USD/candles?start=2021-10-18T19:00:00Z&end=2021-10-19T20:00:00Z&granularity=300

Did not receieve OK response from Coinbase API:https://api.pro.coinbase.com/products/ZRX-USD/candles?start=2021-11-06T13:00:00Z&end=2021-11-07T15:00:00Z&granularity=300
Did not receieve OK response from Coinbase API:https://api.pro.coinbase.com/products/DOGE-USD/candles?start=2021-11-06T13:00:00Z&end=2021-11-07T15:00:00Z&granularity=300
Did not receieve OK response from Coinbase API:https://api.pro.coinbase.com/products/PLA

429 -- Error in 429 https://api.pro.coinbase.com/products/ZRX-USD/candles?start=2022-02-03T03:00:00Z&end=2022-02-04T04:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLA-USD/candles?start=2022-01-09T03:00:00Z&end=2022-01-10T04:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLA-USD/candles?start=2022-01-10T04:00:00Z&end=2022-01-11T05:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLA-USD/candles?start=2022-01-13T07:00:00Z&end=2022-01-14T08:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/DOGE-USD/candles?start=2022-01-22T16:00:00Z&end=2022-01-23T17:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/ZRX-USD/candles?start=2022-02-10T10:00:00Z&end=2022-02-11T11:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/DOGE-USD/candles?start=2022-01-24T18:00:00Z&end=2022-01-25T19:00:00Z&granularity=30

429 -- Error in 429 https://api.pro.coinbase.com/products/1INCH-USD/candles?start=2021-07-05T14:00:00Z&end=2021-07-06T15:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/ENJ-USD/candles?start=2021-06-16T20:00:00Z&end=2021-06-17T21:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/1INCH-USD/candles?start=2021-07-06T15:00:00Z&end=2021-07-07T16:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/ENJ-USD/candles?start=2021-06-17T21:00:00Z&end=2021-06-18T22:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLU-USD/candles?start=2021-06-17T21:00:00Z&end=2021-06-18T22:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/ENJ-USD/candles?start=2021-06-21T00:00:00Z&end=2021-06-22T01:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/ENJ-USD/candles?start=2021-06-22T01:00:00Z&end=2021-06-23T02:00:00Z&granularity=

429 -- Error in 429 https://api.pro.coinbase.com/products/PLU-USD/candles?start=2021-08-15T05:00:00Z&end=2021-08-16T06:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/1INCH-USD/candles?start=2021-09-09T05:00:00Z&end=2021-09-10T06:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/ENJ-USD/candles?start=2021-08-11T01:00:00Z&end=2021-08-12T02:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLU-USD/candles?start=2021-08-17T07:00:00Z&end=2021-08-18T08:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/1INCH-USD/candles?start=2021-09-11T07:00:00Z&end=2021-09-12T08:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/ENJ-USD/candles?start=2021-08-14T04:00:00Z&end=2021-08-15T05:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/1INCH-USD/candles?start=2021-09-12T08:00:00Z&end=2021-09-13T09:00:00Z&granularit

429 -- Error in 429 https://api.pro.coinbase.com/products/PLU-USD/candles?start=2021-11-21T04:00:00Z&end=2021-11-22T05:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/1INCH-USD/candles?start=2021-12-09T22:00:00Z&end=2021-12-10T23:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/ENJ-USD/candles?start=2021-11-28T11:00:00Z&end=2021-11-29T12:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/ENJ-USD/candles?start=2021-11-29T12:00:00Z&end=2021-11-30T13:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLU-USD/candles?start=2021-11-25T08:00:00Z&end=2021-11-26T09:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/1INCH-USD/candles?start=2021-12-14T02:00:00Z&end=2021-12-15T03:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/PLU-USD/candles?start=2021-11-27T10:00:00Z&end=2021-11-28T11:00:00Z&granularity=

429 -- Error in 429 https://api.pro.coinbase.com/products/AAVE-USD/candles?start=2021-08-26T16:00:00Z&end=2021-08-27T17:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/DOT-USD/candles?start=2021-08-29T19:00:00Z&end=2021-08-30T20:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/AAVE-USD/candles?start=2021-08-28T18:00:00Z&end=2021-08-29T19:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/DOT-USD/candles?start=2021-08-30T20:00:00Z&end=2021-08-31T21:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/MLN-USD/candles?start=2021-08-16T06:00:00Z&end=2021-08-17T07:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/AAVE-USD/candles?start=2021-08-31T21:00:00Z&end=2021-09-01T22:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/MLN-USD/candles?start=2021-08-18T08:00:00Z&end=2021-08-19T09:00:00Z&granularity=3

429 -- Error in 429 https://api.pro.coinbase.com/products/MLN-USD/candles?start=2021-10-17T18:00:00Z&end=2021-10-18T19:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/AAVE-USD/candles?start=2021-10-31T07:00:00Z&end=2021-11-01T08:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/AAVE-USD/candles?start=2021-11-01T08:00:00Z&end=2021-11-02T09:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/MLN-USD/candles?start=2021-10-21T22:00:00Z&end=2021-10-22T23:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/AAVE-USD/candles?start=2021-11-02T09:00:00Z&end=2021-11-03T10:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/DOT-USD/candles?start=2021-11-06T13:00:00Z&end=2021-11-07T15:00:00Z&granularity=300

429 -- Error in 429 https://api.pro.coinbase.com/products/AAVE-USD/candles?start=2021-11-05T12:00:00Z&end=2021-11-06T13:00:00Z&granularity=

In [2]:
import glob
# 1. list all text files in the directory
rel_filepaths = glob.glob("*.csv")

# 2. (optional) create a function to read the number of rows in a file
def count_rows(filepath):
  res = 0
  f = open(filepath, 'r')
  res = len(f.readlines())
  f.close()
  return res

# 3. iterate over your files and use the count_row function
ttotal=0
for filepath in rel_filepaths:  
  filecount = count_rows(filepath) 
  print(f'{filepath},{filecount}')
  ttotal += filecount
print(f'Total count:0{ttotal}')


Coinbase_1INCH-USD_dailydata.csv,78672
Coinbase_AAVE-USD_dailydata.csv,81397
Coinbase_ACH-USD_dailydata.csv,63132
Coinbase_ADA-USD_dailydata.csv,81403
Coinbase_AGLD-USD_dailydata.csv,44535
Coinbase_ALCX-USD_dailydata.csv,29442
Coinbase_ALGO-USD_dailydata.csv,81417
Coinbase_AMP-USD_dailydata.csv,78692
Coinbase_ANKR-USD_dailydata.csv,81415
Coinbase_API3-USD_dailydata.csv,24500
Coinbase_ARPA-USD_dailydata.csv,39539
Coinbase_ASM-USD_dailydata.csv,38664
Coinbase_ATOM-USD_dailydata.csv,81417
Coinbase_AUCTION-USD_dailydata.csv,40461
Coinbase_AVAX-USD_dailydata.csv,46388
Coinbase_AVT-USD_dailydata.csv,7723
Coinbase_AXS-USD_dailydata.csv,60477
Coinbase_BADGER-USD_dailydata.csv,39735
Coinbase_BAL-USD_dailydata.csv,77322
Coinbase_BAND-USD_dailydata.csv,78037
Coinbase_BAT-USD_dailydata.csv,81108
Coinbase_BCH-USD_dailydata.csv,81416
Coinbase_BICO-USD_dailydata.csv,27875
Coinbase_BLZ-USD_dailydata.csv,24957
Coinbase_BNT-USD_dailydata.csv,66842
Coinbase_BOND-USD_dailydata.csv,72505
Coinbase_BTC-USD_d